<h1> Pandas Data Type Inference and Conversion (Backend Task) </h1>

In [25]:
import pandas as pd
import numpy as np

In [26]:
df = pd.read_csv('annual.csv', parse_dates=True , infer_datetime_format=True, low_memory=False)
df


/var/folders/xm/pvqw3jjj1zvb8k4_8681y4yc0000gn/T/ipykernel_93389/1581276647.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('annual.csv', parse_dates=True , infer_datetime_format=True, low_memory=False)


,year,industry_code_ANZSIC,industry_name_ANZSIC,rme_size_grp,variable,value,unit
0,2011,A,"Agriculture, Forestry and Fishing",a_0,Activity unit,46134,COUNT
1,2011,A,"Agriculture, Forestry and Fishing",a_0,Rolling mean employees,0,COUNT
2,2011,A,"Agriculture, Forestry and Fishing",a_0,Salaries and wages paid,279,DOLLARS(millions)
3,2011,A,"Agriculture, Forestry and Fishing",a_0,"Sales, government funding, grants and subsidies",8187,DOLLARS(millions)
4,2011,A,"Agriculture, Forestry and Fishing",a_0,Total income,8866,DOLLARS(millions)
...,...,...,...,...,...,...,...
17023,2021,all,All Industries,j_Grand_Total,Total income,757504,DOLLARS(millions)
17024,2021,all,All Industries,j_Grand_Total,Total expenditure,654404,DOLLARS(millions)
17025,2021,all,All Industries,j_Grand_Total,Operating profit before tax,85116,DOLLARS(millions)
17026,2021,all,All Industries,j_Grand_Total,Total assets,2512677,DOLLARS(millions)


In [27]:
df.isna().sum()

year                    0
industry_code_ANZSIC    0
industry_name_ANZSIC    0
rme_size_grp            0
variable                0
value                   0
unit                    0
dtype: int64

In [28]:
## helper functions
def is_numeric(col: pd.Series):
    series = col.dropna()
    try:
    # Attempt to convert to numeric first
        return not all(pd.isna(pd.to_numeric(series, errors='coerce')))
    except:
        return False

def is_float(col: pd.Series):
    series = col.dropna()
    try:
        if (pd.to_numeric(series, errors='coerce') % 1 == 0).all():
            return True
    except:
        return False

def is_bool(col: pd.Series):
    series = col.dropna()
    if any(isinstance(value, bool) for value in series):
        return True
    
    # Convert strings "True" and "False" to actual boolean values
    series_lower = series.apply(lambda x: x.lower() if isinstance(x, str) else x)

    # Check if all values are boolean after converting string representations
    if any(value in {"true", "False"} for value in series_lower):
        return True
    
    return False

def is_category(col: pd.Series):
    series = col.dropna()
    if len(series.unique()) / len(series) <= 0.5:  # Example threshold for categorization
        return True
    return False

def is_datetime(col: pd.Series):
    series = col.dropna()
    return any(series.apply(lambda x: pd.to_datetime(x, errors='coerce') is not pd.NaT))

def is_timedelta(col: pd.Series):
    series = col.dropna()
    return any(series.apply(lambda x: pd.to_timedelta(x, errors='coerce') is not pd.NaT))

def is_complex(col: pd.Series):
    series = col.dropna()
    return any(series.apply(lambda x: isinstance(x, complex) or isinstance(x, str) and '+' in x and 'j' in x))

 # Function to check if a value is null or NaN
def is_null_or_nan(value):
    return pd.isnull(value) or pd.isna(value)

def clean_column(col: pd.Series):
    series = col.dropna()
    series = series.drop_duplicates()
    return series


In [29]:
## infer dtype of a column
def infer_col_type(col: pd.Series):
    if is_numeric(col):
        if is_float(col=col):
            return "decimal"
        else:
            return "int"
    else:
        if is_bool(col=col):
            return "boolean"
        elif is_category(col=col):
            return "category"
        elif is_datetime(col=col):
            return "datetime"
        elif is_complex(col=col):
            return "complex"
        elif is_timedelta(col=col):
            return "timedelta"
        elif is_datetime(col=col):
            return "datetime"
    
    return "object"
data_types_all = []           
for col in df.columns:
    data_types_all.append(infer_col_type(df[col]))
    print(infer_col_type(df[col]))
    
    
print(data_types_all)

decimal
category
category
category
category
int
category
['decimal', 'category', 'category', 'category', 'category', 'int', 'category']


In [30]:
print("\nData types after inference:")
print(df.dtypes)


Data types after inference:
year                     int64
industry_code_ANZSIC    object
industry_name_ANZSIC    object
rme_size_grp            object
variable                object
value                   object
unit                    object
dtype: object


In [10]:
def infer_and_convert_data_types(df):
    for col in df.columns:
        # Attempt to convert to numeric first
        df_converted = pd.to_numeric(df[col], errors='coerce')
        if not df_converted.isna().all():  # If at least one value is numeric
            df[col] = df_converted
            continue

        # Attempt to convert to datetime
        try:
            df[col] = pd.to_datetime(df[col], format='%m/%d/%Y')
            continue
        except (ValueError, TypeError):
            pass
        

        # Check if the column should be categorical
        if len(df[col].unique()) / len(df[col]) < 0.5:  # Example threshold for categorization
            df[col] = pd.Categorical(df[col])

    return df

df = infer_and_convert_data_types(df)

print("\nData types after inference:")
print(df.dtypes)


Data types after inference:
Name                 object
Birthdate    datetime64[ns]
Score               float64
Grade              category
dtype: object


In [16]:
df

,Name,Birthdate,Score,Grade
0,Alice,1/01/1990,90,A
1,Bob,2/02/1991,75,B
2,Charlie,3/03/1992,85,A
3,David,4/04/1993,70,B
4,Eve,5/05/1994,Not Available,A


In [ ]:
##### CODE DUMP : DO NOT RUN 

In [235]:
## helper functions
def is_numeric(col: pd.Series):
    series = clean_column(col)
    try:
    # Attempt to convert to numeric first
        return not all(pd.isna(pd.to_numeric(series, errors='coerce')))
    except:
        return False

def is_float(col: pd.Series):
    series = clean_column(col)
    try:
        if (pd.to_numeric(series, errors='coerce') % 1 == 0).all():
            return True
    except:
        return False

def is_bool(col: pd.Series):
    series = clean_column(col)
    if any(isinstance(value, bool) for value in series):
        return True
    
    # Convert strings "True" and "False" to actual boolean values
    series_lower = series.apply(lambda x: x.lower() if isinstance(x, str) else x)

    # Check if all values are boolean after converting string representations
    if any(value in {"true", "False"} for value in series_lower):
        return True
    
    return False

def convert_to_bool(col: pd.Series) -> pd.Series:
    series = col
    if any(isinstance(value, bool) for value in series):
        return series
    series = series.fillna("false")
    # Convert strings "True" and "False" to actual boolean values
    series = series.map(lambda x: True if x.lower() == 'true' else (False if x.lower() == 'false' else x))
    return series

def convert_to_complex(col: pd.Series) -> pd.Series:
    series = col
    if any(isinstance(value, complex) for value in series):
        return series
    series = series.fillna(complex(0))
    # Convert strings "True" and "False" to actual boolean values
    series = series.map(lambda x: complex(x) if (isinstance(x, str) and ('+' in x or '-' in x) and 'j' in x) or (is_numeric(pd.Series([x]))) else complex(0))
    return series

def is_category(col: pd.Series):
    series = col.dropna()
    if len(series.unique()) / len(series) <= 0.5:  # Example threshold for categorization
        return True
    return False

def is_datetime(col: pd.Series):
    series = clean_column(col)
    return any(series.apply(lambda x: pd.to_datetime(x, errors='coerce') is not pd.NaT))

def is_timedelta(col: pd.Series):
    series = clean_column(col)
    return any(series.apply(lambda x: pd.to_timedelta(x, errors='coerce') is not pd.NaT))

def is_complex(col: pd.Series):
    series = clean_column(col)
    return any(series.apply(lambda x: isinstance(x, complex) or isinstance(x, str) and '+' in x and 'j' in x))

 # Function to check if a value is null or NaN
def is_null_or_nan(value):
    return pd.isnull(value) or pd.isna(value)

def clean_column(col: pd.Series):
    series = col.dropna()
    series = series.drop_duplicates()
    return series


In [ ]:
## test block
# any(pd.isna(pd.to_numeric(pd.Series([1,2]), errors='coerce')))
# all(pd.isna(pd.to_numeric(pd.Series(["noo","1"]), errors='coerce')))
# is_numeric(pd.Series([pd.NA,"lala"]))
# is_complex(pd.Series(['3+4jj', '5-6jj', 'Not available', '7+8jj']))
is_timedelta(pd.Series(["1 days 00:00:00"]))
convert_to_bool(pd.Series(["true","false", pd.NA]))
convert_to_complex(pd.Series(['3+4j', '5-6j', 'Not available']))
pd.Categorical([1,"d", "c"])

In [1]:

## infer dtype of a column
def infer_col_type(col: pd.Series):
    if is_numeric(col):
        if is_float(col=col):
            return pd.to_numeric(col, downcast="float", errors='coerce')
        else:
            return pd.to_numeric(col, downcast="integer",errors='coerce')
    else:
        if is_bool(col=col):
            return convert_to_bool(col=col)
        elif is_category(col=col):
            return pd.Categorical(col)
        elif is_datetime(col=col):
            return pd.to_datetime(col, errors='coerce')
        elif is_complex(col=col):
            return convert_to_complex(col)
        elif is_timedelta(col=col):
            return pd.to_timedelta(col, errors='coerce')
    
    return "object"
            
for col in df.columns:
    df[col] = infer_col_type(df[col])
df.info()

NameError: name 'pd' is not defined